Import Data CSV

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt

df = pd.read_csv("C:/Users/hp/Downloads/ELearning Myskill 6 Bulan/Data Analyst/Python For Data Analysis/Project - User Retention Analysis/Online_Retail_Data.csv")
display(df.isna().sum())
display(df.head())
display(df.info())
display(df.shape)
display(df.describe())

plt.figure(figsize=(8,4))
sns.boxplot(x=df['quantity'])
plt.show()

Data Cleansing dan Buat Kolom Tahun-Bulan

In [108]:

df_clean = df.copy()
df_clean['order_date'] = df_clean['order_date'].astype('datetime64[ns]')
df_clean['year_month'] = df_clean['order_date'].dt.to_period('M')
#menghapus missing value pada kolom customer_id
df_clean = df_clean.dropna(subset=['customer_id'])
#menghapus missing value pada kolom product_name
df_clean = df_clean.dropna(subset=['product_name'])
#karena tidak hanya missing values yg perlu dihapus, tapi juga perlu juga disamakan katanya
#maka, mengubah product_name menjadi huruf kecil semua
df_clean['product_name'] = df_clean['product_name'].str.lower()
# mengubah nilai quantity yang negatif menjadi positif karena nilai negatif tersebut hanya menandakan order tersebut cancelled
df_clean['quantity'] = df_clean['quantity'].abs()
# menghapus semua baris dengan product_code atau product_name yang berisi 'test;
df_clean = df_clean[(~df_clean['product_code'].str.lower().str.contains('test')) |
                    (~df_clean['product_name'].str.contains('test '))]
# menghapus baris dengan price bernilai negatif
df_clean = df_clean[df_clean['price']>0]
# membuat nilai amount, yaitu perkalian antara quantity dan price
df_clean['amount'] = df_clean['quantity'] * df_clean['price']
# membuat kolom order_status dengan nilai 'cancelled' jika order_id diawali dengan huruf 'c' dan 'delivered' jika order_id tanpa awalan huruf 'c'
df_clean['order_status'] = np.where(df_clean['order_id'].str[:1]=='C', 'cancelled', 'delivered')
# mengganti product_name dari product_code yang memiliki beberapa product_name dengan salah satu product_name-nya yang paling sering muncul
most_freq_product_name = (
    df_clean.groupby(['product_code','product_name'], as_index=False)
    .agg(order_cnt=('order_id','nunique'))
    .sort_values(['product_code','order_cnt'], ascending=[True,False])
)
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])
df_clean = df_clean.merge(most_freq_product_name.rename(columns={'product_name':'most_freq_product_name'}), how='left', on='product_code')
df_clean['product_name'] = df_clean['most_freq_product_name']
df_clean = df_clean.drop(columns='most_freq_product_name')
# mengkonversi customer_id menjadi string
df_clean['customer_id'] = df_clean['customer_id'].astype(str)
# menghapus outlier
from scipy import stats
df_clean = df_clean[(np.abs(stats.zscore(df_clean[['quantity','amount']]))<3).all(axis=1)]
df_clean = df_clean.reset_index(drop=True)
df_clean
display(df_clean.info())
display(df_clean.head())
display(df_clean.tail())

#plt.figure(figsize=(8,4))
#sns.boxplot(x=df_clean['quantity','amount'])
#plt.show()

#display(df_clean['product_name'].nunique())
#display(df_clean.isna().sum())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358469 entries, 0 to 358468
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      358469 non-null  object        
 1   product_code  358469 non-null  object        
 2   product_name  358469 non-null  object        
 3   quantity      358469 non-null  int64         
 4   order_date    358469 non-null  datetime64[ns]
 5   price         358469 non-null  float64       
 6   customer_id   358469 non-null  object        
 7   year_month    358469 non-null  period[M]     
 8   amount        358469 non-null  float64       
 9   order_status  358469 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5), period[M](1)
memory usage: 27.3+ MB


None

,order_id,product_code,product_name,quantity,order_date,price,customer_id,year_month,amount,order_status
0,C493411,21539,red retrospot butter dish,1,2010-01-04 09:43:00,4.25,14590.0,2010-01,4.25,cancelled
1,493414,21844,red retrospot mug,36,2010-01-04 10:28:00,2.55,14590.0,2010-01,91.80,delivered
2,493414,21533,retro spot large milk jug,12,2010-01-04 10:28:00,4.25,14590.0,2010-01,51.00,delivered
3,493414,37508,new england ceramic cake server,2,2010-01-04 10:28:00,2.55,14590.0,2010-01,5.10,delivered
4,493414,35001G,hand open shape gold,2,2010-01-04 10:28:00,4.25,14590.0,2010-01,8.50,delivered


,order_id,product_code,product_name,quantity,order_date,price,customer_id,year_month,amount,order_status
358464,539988,84380,set of 3 butterfly cookie cutters,1,2010-12-23 16:06:00,1.25,18116.0,2010-12,1.25,delivered
358465,539988,84849D,hot baths soap holder,1,2010-12-23 16:06:00,1.69,18116.0,2010-12,1.69,delivered
358466,539988,84849B,fairy soap soap holder,1,2010-12-23 16:06:00,1.69,18116.0,2010-12,1.69,delivered
358467,539988,22854,cream sweetheart egg holder,2,2010-12-23 16:06:00,4.95,18116.0,2010-12,9.90,delivered
358468,539988,47559B,tea time oven glove,2,2010-12-23 16:06:00,1.25,18116.0,2010-12,2.50,delivered
